In [1]:
import pandas as pd
import numpy as np
import requests
import json
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

# Dúvidas gerais

- Q2:
    - Os retornos deveriam ser acumulados?
    - Como mostrar a diferença das duas medidas da melhor forma?
- Q3:
    - "Cada ação deve ter seus gráficos apresentados como output de uma única célula." se interpretei direito cada ação fica plotada separada, o que seria ruim para comparar as duas, é isso mesmo?
- Q4:
    - Os retornos mensais e trimestrais pega a média mesmo ou o retorno acumulado até o final do período?

# Q1. 
- Usando o  python  (sugerimos o  pacote  yfinance  ou  o  pandas), importe  os  dados  de  preço  de fechamento de 01/11/2017 até 01/01/2022 das ações que compõem o indice IBRX 50. Preencha dados faltantes de cada série com as médias dos vizinhos. Mostre que o DataFrame resultante não temvalores vazios.

## 1.1 Getting the current stocks in IBRX 50

### - 1.1.1 From Infomoney

In [2]:
ibrx50_im_url = 'https://www.infomoney.com.br/cotacoes/b3/indice/ibrx50/'
ibrx50_im = pd.read_html(ibrx50_im_url, decimal=',', thousands='.')[2:]
ibrx50_im = pd.concat(ibrx50_im).reset_index(drop=True)
ibrx50_im

,Ativo,Último (R$),Var. Dia (%),Val. Min (R$),Val. Máx (R$),Data
0,GGBR4,25.82,0.15,25.41,26.12,17:07 11/10
1,BRKM5,27.89,20.39,30.20,34.29,17:07 11/10
2,ELET6,48.17,0.00,47.84,48.52,17:07 11/10
3,USIM5,7.65,1.18,7.54,7.80,17:07 11/10
4,NTCO3,13.16,0.30,12.90,13.48,17:07 11/10
5,RENT3,65.34,0.69,64.98,66.63,17:07 11/10
6,CSAN3,15.40,1.69,15.25,16.02,17:07 11/10
7,EQTL3,28.36,1.02,28.28,28.80,17:11 11/10
8,RAIL3,18.23,4.22,18.07,19.03,17:07 11/10
9,MULT3,25.17,0.00,24.95,25.43,17:07 11/10


In [3]:
# Saving only tickers
stocks_im = ibrx50_im['Ativo'].tolist()
#stocks_im

In [ ]:
#ibrx50_im.to_excel('ibrx50_im.xlsx', index = False)

### 1.1.2 From B3 - Alternative source (Actually the main)


In [4]:
# 
url = 'https://sistemaswebb3-listados.b3.com.br/indexProxy/indexCall/GetPortfolioDay/eyJpbmRleCI6IklCWEwiLCJsYW5ndWFnZSI6InB0LWJyIn0='
r = requests.get(url)
ibrx50_b3 =  pd.json_normalize(json.loads(r.content), record_path =['results'])
# Treating data - Just by habit, it is not necessary because I only need the ticker/ cod column
ibrx50_b3.drop(columns=['segment', 'partAcum'], inplace=True) # Treating data, but not really necessary
ibrx50_b3['part'] = ibrx50_b3['part'].apply(lambda x: float(x.replace(',', '.'))/100) # Also not necessary in this case
ibrx50_b3['theoricalQty'] = ibrx50_b3['theoricalQty'].apply(lambda x: int(x.replace('.', ''))) # Same as above
ibrx50_b3.sort_values(by='part', ascending=False, inplace=True, ignore_index=True) #... and so on
ibrx50_b3


,cod,asset,type,part,theoricalQty
0,VALE3,VALE,ON NM,0.17530,4547296398
1,PETR4,PETROBRAS,PN N2,0.07905,4566441248
2,ITUB4,ITAUUNIBANCO,PN EJ N1,0.07254,4799907051
3,PETR3,PETROBRAS,ON N2,0.07135,3701760571
4,BBDC4,BRADESCO,PN EJ N1,0.05439,5156077326
5,ELET3,ELETROBRAS,ON N1,0.04829,2021139464
6,B3SA3,B3,ON NM,0.04443,5901731302
7,ABEV3,AMBEV S/A,ON,0.03449,4386652506
8,RENT3,LOCALIZA,ON NM,0.03085,893978241
9,BBAS3,BRASIL,ON NM,0.02915,1420731069


In [ ]:
#ibrx50_b3.to_excel('ibrx50_b3.xlsx', index = False)

In [5]:
# Subsetting only tickers and adding '.SA' to match yfinance format
tickers = [ticker + '.SA' for ticker in ibrx50_b3['cod']]
tickers

['VALE3.SA',
 'PETR4.SA',
 'ITUB4.SA',
 'PETR3.SA',
 'BBDC4.SA',
 'ELET3.SA',
 'B3SA3.SA',
 'ABEV3.SA',
 'RENT3.SA',
 'BBAS3.SA',
 'WEGE3.SA',
 'ITSA4.SA',
 'SUZB3.SA',
 'HAPV3.SA',
 'BPAC11.SA',
 'EQTL3.SA',
 'LREN3.SA',
 'GGBR4.SA',
 'RADL3.SA',
 'RDOR3.SA',
 'JBSS3.SA',
 'PRIO3.SA',
 'RAIL3.SA',
 'VBBR3.SA',
 'BBSE3.SA',
 'CSAN3.SA',
 'TOTS3.SA',
 'BRFS3.SA',
 'ASAI3.SA',
 'MGLU3.SA',
 'CCRO3.SA',
 'KLBN11.SA',
 'ELET6.SA',
 'AMER3.SA',
 'NTCO3.SA',
 'EMBR3.SA',
 'BRKM5.SA',
 'RRRP3.SA',
 'BRML3.SA',
 'CSNA3.SA',
 'MULT3.SA',
 'BRAP4.SA',
 'VIIA3.SA',
 'AZUL4.SA',
 'LWSA3.SA',
 'USIM5.SA',
 'MRFG3.SA',
 'PETZ3.SA',
 'CVCB3.SA',
 'CASH3.SA']

## - 1.2 Importing closing prices from 01/11/2017 to 01/01/2022

In [6]:
# Actually getting Adjusted closing prices from yfinance
prices = yf.download(tickers, start='2017-11-01', end='2022-01-01', rounding=True)['Adj Close']
prices.columns = [col.rstrip('.SA') for col in prices.columns] # Removing '.SA' from column names
prices


KeyboardInterrupt: 

In [ ]:
# MAIORES DADOS FALTANTES - Naturalmente são as ações que não haviam capital aberto em todo período
prices.isna().sum().sort_values(ascending=False)

In [ ]:
# Visualizando
plt.figure(figsize=(15, 10))
prices.isna().sum().plot(kind='bar')

## - 1.3 Filling na values with the mean of the neighbors

In [ ]:
# Com list comprehension - perde um pouco de legibilidade
# Copiei para outro dataframe para não alterar o original, mas poderia ser mais simples
prices_filled = pd.DataFrame([prices.loc[:, col].fillna(prices.loc[:, col].mean()) for col in prices.columns]).T
prices_filled

In [ ]:
# Com for loop, bem intuitivo (pra mim), e quase a mesma coisa do anterior
# Rodar esse ou o de cima, escolhi o método de list comprehension e armazenar em outa variável

'''
for col in prices.columns:
    prices.loc[:, col].fillna(prices.loc[:, col].mean(), inplace=True)
'''

## - 1.4 Visualizing treated na values


In [ ]:
prices_filled.isna().sum().sort_values(ascending=False)

In [ ]:
# Visualizando
prices_filled.isna().sum().plot(kind='bar')


# Q2. 
- Em uma célula mostre os últimos 12 dias de preços, ordenados do mais recente para o mais antigo. Nas 2 células seguintes mostre os retornos diários para os mesmos dias, na primeira calculados usando diferença de logaritmos e na segunda a diferença percentual. Mostre a diferença das duas medidas.

## - 2.1 Last 12 days from newest to oldest

In [ ]:
# I didn't keep the change in the code, just showing the result
last12 = prices_filled.tail(15)
last12.sort_values(by='Date', ascending=False)

In [ ]:
# Retorno Logarítmico - Confirmar fórmula
log_return = np.log(last12/ last12.shift(1))
log_return


In [ ]:
# Retorno Percentual
pct_return = last12.pct_change()
pct_return

Não entendi exatamente como era para mostrar a diferença dos dois valores além do output, então vou pegar uma amostra da última linha de ambos e comparar, poderia ter usado o retorno acumulado também

In [ ]:
# Última linha retorno logarítmico  
log_return.iloc[-2:-1]

In [ ]:
# Última linha retorno percentual
pct_return.iloc[-2:-1]

In [ ]:
#
# Comparing result from log and pct returns and calculating the difference
delta = []
delta.append(log_return.iloc[-2:-1])
delta.append(pct_return.iloc[-2:-1])
delta = pd.concat(delta)
delta.loc['diff'] = delta.iloc[0] - delta.iloc[1]
delta


In [ ]:
# Showing in a graph
plt.figure(figsize=(15, 10))
sns.barplot(x=delta.loc['diff'].index, y=delta.loc['diff'].values)
plt.xticks(rotation=45)
None

In [ ]:
# Escolhendo uma ação e plotando o gráfico com retorno logarítmo e retorno percentual
plt.figure(figsize=(15, 10))
plt.plot(log_return['AZUL4'])
plt.plot(pct_return['AZUL4'])
plt.legend(['Log Return', 'Pct Return']) 
plt.title('Comparação métodos de Retorno - Logarítmo e Percentual')
None

# Q3. 
- Escolha 2 ações das 50 estudadas, que tenham sido negociadas desde pelo menos 01/11/2017, deixe claro quais você escolheu. Apresente graficamente o histórico de preços, de variação e a distribuição dos retornos diários delas. Cada ação deve ter seus gráficos apresentados como output de uma única célula.

## - 3.1 Choosing the stocks - The idea is to choose automaticaly the stocks with the most and least returns

- First I'll drop the holiday date and then drop the columns with na values, the ones that don't have data since 2017-11-01
- Second calculate the accumulated returns and filter the ones with the most and least returns

In [ ]:
# Desde a primeira visualização de dados nulos deu pra perceber todas as ações vieram com ao menos um valor nulo
# Acabou vindo com a sexta feira Santa de 2021
prices[prices['WEGE3'].isna()]

In [ ]:
# Fazendo um cópia, retirando da data com problema e dropando as colunas com dados faltantes
prices_dropped = prices.copy()
prices_dropped.drop(index='2021-04-02', inplace=True) # dava pra pegar o index de forma automática tb
prices_dropped.dropna(axis=1, inplace=True) # Dropando colunas sem dados desde o início
prices_dropped

In [ ]:
# Sobraram 41 ações com dados desde o início
# escolher automaticamente as duas ações com maior e pior retorno e que estão desde o começo do período

# Já dá para ver que PETRO RIO e CVC são respectivamente a melhor e pior ação baseado em retorno percentual acumulado
simple_return = (prices_dropped.pct_change() + 1).cumprod()
simple_return.iloc[-1].sort_values(ascending=False)

In [ ]:
# Maior Retorno de forma automática - transpondo e manipulando o df para ficar mais fácil
teste = simple_return.T.reset_index().rename(columns={'index': 'ticker'})

# Ação com maior retorno	
max_stock = teste[teste.iloc[:, -1] == teste.iloc[:, -1].max()]['ticker'].values[0]
max_stock

In [ ]:
# Ou com outro método mais fácil dependendo da perspectiva
teste.nlargest(1, columns=teste.columns[-1])['ticker'].values[0]

In [ ]:
# Ação com menor retorno
min_stock = teste[teste.iloc[:, -1] == teste.iloc[:, -1].min()]['ticker'].values[0]
min_stock

In [ ]:
# Ou com outro método mais fácil dependendo da perspectiva
teste.nsmallest(1, columns=teste.columns[-1])['ticker'].values[0]

## - 3.2 Chosen Stocks:

- PRIO3
- CVCB3

### Graphically presenting the history, variation and distribution of the daily returns

Apresente graficamente o histórico de preços, de variação e a distribuição dos retornos diários delas. Cada ação deve ter seus gráficos apresentados como output de uma única célula.

In [ ]:
prices_filled[[max_stock, min_stock]]


In [ ]:
prices_filled[[max_stock, min_stock]].plot(figsize=(15, 10), title='Histotico de Preços')

In [ ]:
prices_filled[[max_stock, min_stock]].pct_change().plot(figsize=(15, 10), title='Variação de Preços')

In [ ]:
(prices_filled[[max_stock, min_stock]].pct_change() + 1).cumprod().plot(figsize=(15, 10), title='Variação Acumulada de Preços')

In [ ]:
# Melhorar isso aqui depois
plt.hist(prices_filled[max_stock].pct_change().dropna(), bins=150, label=max_stock, alpha=0.5)
plt.hist(prices_filled[min_stock].pct_change().dropna(), bins=150, label=min_stock, alpha=0.5)
plt.title('Histograma de Retorno Percentual')
plt.legend()
None

# Q4.
- Crie dataframes com os retornos mensais e trimestrais das 2 ações e a variação do IBOVESPA. A partir desses mostre a média, mediana, variancia, 1º e 3º quartil para o retorno trimestrais e mensais das ações.

## - 4.1 Getting Ibovespa data

In [ ]:
# Dados Ibov
ibov = pd.DataFrame(yf.download('^BVSP', start='2017-11-01', end='2022-01-01', rounding=True)['Adj Close'])
ibov.rename(columns={'Adj Close': 'IBOV'}, inplace=True)
ibov

## - 4.2 Merging dfs

In [ ]:
# Merging dataframes
merged = pd.merge(prices_filled[[max_stock, min_stock]], ibov, left_index=True, right_index=True)
merged

## - 4.3 Month Returns

In [ ]:
#resample
month_returns = merged.pct_change().resample('M').mean()
month_returns

In [ ]:
# Estatística descritiva com todos os valores pedidos
month_returns.describe()

In [ ]:
# Boxplot
sns.boxplot(data=month_returns, whis=10)

# Plot the y-axis on a log scale
#plt.yscale('log')

# Remove unneeded lines and label axes
#sns.despine(left=True, bottom=True)
plt.xlabel('Tickers')
plt.ylabel('Percentual Returns')
plt.show()

## - 4.4 Quarter Returns

In [ ]:
quarter_returns = merged.pct_change().resample('Q').mean()
quarter_returns

In [ ]:
#Estatística descritiva com todos os valores pedidos
quarter_returns.describe()

In [ ]:
# Boxplot
sns.boxplot(data=quarter_returns, whis=10)

# Plot the y-axis on a log scale
#plt.yscale('log')

# Remove unneeded lines and label axes
#sns.despine(left=True, bottom=True)
plt.xlabel('Tickers')
plt.ylabel('Percentual Returns')
plt.show()


# Q5.
- Utilizando os dados dessas mesmas 2 ações implemente uma estratégia de trade para cada uma delas usando Índice de Força Relativa, com janela de 21 dias úteis e pontos de entrada de 75 e 25. Mostre o resultado das estratégias para o período de 01/01/2018 a 31/12/2021.

In [ ]:
#rsi


# Q6.
- Apresente  as  métricas e  gráficos  que  achar  relevantes  sobre  as  estratégias  da  Q5,  comparando  as duas ações e avaliando cada estratégia em comparação ao desempenho do IBOVESPA.

Bonus. Comente e, se possível, implementecomo seria possível melhorar as estratégias testadas.Lembre-se,avaliaremos também a clareza e compreensibilidadedo código, utilize comentários a vontade para facilitar o entendimento. Para poder avaliar o código precisamos ser capazes de executá-lo, garantaque todas as dependências são facilmente encontradas e disponíveis.